In [1]:
# lets take a look at the kappa scores

In [2]:
from collections import Counter, defaultdict
from prodigy.components.db import connect
from prodigy import set_hashes
import pandas as pd

KeyboardInterrupt: 

In [ ]:
input_hashes = defaultdict(list)
counts = Counter()
db = connect()
answer_dict = defaultdict(lambda: {'accept':0, 'reject':0, 'ignore':0})
examples = db.get_dataset("n2c2_assessment")
examples = [set_hashes(eg) for eg in examples]
for eg in examples:
    session = eg["_session_id"].split("-")[1] if "_session_id" in eg else "n/a"
    answer_dict[session][eg['answer']] +=1
    counts[session] += 1
print(counts)

In [ ]:
annotations_df = pd.DataFrame(examples)

In [ ]:
'''DC = annotations_df[annotations_df['_annotator_id'] == 'n2c2_planSubsection-dc2249']
AG = annotations_df[annotations_df['_annotator_id'] == 'n2c2_planSubsection-asg64']'''

In [ ]:
DC = annotations_df[annotations_df['_annotator_id'] == 'n2c2_assessment-dc2249']
AG = annotations_df[annotations_df['_annotator_id'] == 'n2c2_assessment-asg64']

In [ ]:
input_hashes = list(set(DC['_input_hash'].tolist() + AG['_input_hash'].tolist()))

In [ ]:
input_hashes

In [ ]:
len(input_hashes)

In [ ]:
import nltk

In [ ]:
lst_AG

In [467]:
from collections import defaultdict
assessment_labels = {x:idx for idx, x in enumerate(['primary_problem','secondary_problem','primary_symptom','primary_sign'])}
planSubsection_labels = {x:idx for idx, x in enumerate(['problem','complication_related_to_problem','event_related_to_problem','organ_failure_related_to_problem'])}
IAA_f1_score = {}
saved_annotations = []
for h in input_hashes:
    AG_df_hash = AG[AG['_input_hash'] == h]
    DC_df_hash = DC[DC['_input_hash'] == h]
    if len(DC_df_hash) == 0 or len(AG_df_hash) == 0:
        continue
    AG_df_hash = AG_df_hash.iloc[0]# get spans
    DC_df_hash = DC_df_hash.iloc[0]# get spans
    # if spans in the df, if spans are not NA and if the len of the span is larger than 0
    ann1 = type(AG_df_hash['spans']) == float or len(AG_df_hash['spans']) == 0
    ann2 = type(DC_df_hash['spans']) == float or len(DC_df_hash['spans']) == 0
    if ann1 or ann2:
        if len(saved_annotations) < 6:
            saved_annotations.append({'DC': AG_df_hash['spans'], 'AG': DC_df_hash['spans'], 'text': DC_df_hash['text']})
        continue
    if len(saved_annotations) < 10:
        saved_annotations.append({'DC': AG_df_hash['spans'], 'AG': DC_df_hash['spans'], 'text': DC_df_hash['text']})
    lst_AG = AG_df_hash['spans']
    lst_DC = DC_df_hash['spans']
    # extract the list elements that have overlap (This may cause some issue where an annotation from a1 is overlapped 2 times by a2)
    # I am going to just count it as another y value
    # so what we want to do is capture the intervals for the spans, then check to see if the spans are different. in 3 ways
    #   i) binary absolute match (EXACT match on spans, no match), 
    #  ii) any overlap (if there is any overlap then its an agreement) and 
    # iii) weighted overlap by jaccard distance/character numbers (amount of overlap is > 50%)
    spans_AG = [[x['start'], x['end'], {'AG': assessment_labels[x['label']]}] for x in lst_AG]
    spans_DC = [[x['start'], x['end'], {'DC': assessment_labels[x['label']]}] for x in lst_DC]
    # now we can update the annotator-label dict with both annotators on the merged
    spans = spans_AG + spans_DC
    if len(spans) == 0:# in the event the spans are empty
        continue
    spans.sort(key=lambda x: x[0])# sort them by the first element
    merged = []
    for interval in spans:
        # if the list of merged intervals is empty or if the current
        # interval does not overlap with the previous, simply append it.
        if not merged or merged[-1][1] < interval[0]:
            merged.append(interval)
        else:
        # otherwise, there is overlap, so we merge the current and previous
        # intervals.
            merged[-1][1] = max(merged[-1][1], interval[1])
            merged[-1][2].update(interval[2])
    lst = []
    for x in merged:
        x[2]['interval'] = [x[0], x[1]]
        lst.append(x[2])
    # test code, if we got here then we should save last dfs
    last_AG = AG_df_hash
    last_DC = DC_df_hash
    tmp_df = pd.DataFrame(lst).set_index('interval').fillna(-1)
    IAA_f1_score[h] = f1_score(tmp_df['AG'], tmp_df['DC'], average='weighted')

    '''for ag_idx, ag in enumerate(lst_AG):
        for dc_idx, dc in enumerate(lst_DC):
            s1 = ag['start']
            s2 = dc['start']
            e1 = ag['end']
            e2 = dc['end']
            # any overlap
            if (s1 <= e2) and (s2 <= e1):
                matches[h]['binary_absolute'].append(True if s1 == s2 and e1 == e2 else False)
                matches[h]['any_overlap'].append(True) # true because we are in this if statement
                matches[h]['levenshtein_distance'].append(nltk.edit_distance(AG_df_hash.iloc[ag_idx,]['text'][s1:e1+1], DC_df_hash.iloc[dc_idx]['text'][s2:e2+1]))
                matches[h]['label_match'].append([lst_AG[ag_idx]['label'], lst_DC[dc_idx]['label']])
                matches[h]['annotator_a_span'].append(ag)
                matches[h]['annotator_b_span'].append(dc)'''

        

In [490]:

[print(x['text']) for x in saved_annotations]

ALTERED MENTAL STATUS (NOT DELIRIUM)
 .H/O CARDIOMYOPATHY, OTHER
 RESPIRATORY FAILURE, ACUTE (NOT ARDS/[**Doctor Last Name 76**])
   This is a 75 yo M with [**Last Name (LF) 7287**], [**First Name3 (LF) **] 15%, afib on coumadin, s/p pacer
   placement, admitted from [**Hospital1 **] with RLL pna and respiratory distress.
Leukocytosis.  Likely secondary to SBO, but will evaluate for
   infectious sources.  Also has infiltrate on CXR, ?aspiration
   pneumonitis versus pneumonia.
   - f/u urine, blood, sputum cultures
   - monitor fever curve, wbc
   - hold off on Abx for now, but consider if febrile or has worsening
   respiratory symptoms
Ms [**Known lastname 11751**] is a 45 yo woman with pmh of DM1, CAD s/p CABG and PCI,
   ESRD s/p living-related kidney transplant in [**10-31**], and recent
   hospitalization for septic arthritis here with acute renal failure,
   acute diastolic heart failure with newly diagnosed severe MR and
   pulmonary hypertension.
84 yoM with a history of CAD,

[None, None, None, None, None, None, None, None, None, None]

In [476]:
import json
with open('random_10_samples_n2c2.json', 'w') as outfile:
    json.dump(saved_annotations, outfile)

In [1]:
np.mean(list(IAA_f1_score.values()))

NameError: name 'np' is not defined

In [440]:
last_AG['spans']

[{'start': 2,
  'end': 7,
  'text': 'STEMI',
  'source': '/home/kl533/n2c2/models/plan_subsection_model/model-best-fixed/',
  'input_hash': 1561130997,
  'token_start': 1,
  'token_end': 1,
  'label': 'problem'},
 {'start': 23,
  'end': 54,
  'token_start': 6,
  'token_end': 15,
  'label': 'complication_related_to_problem'},
 {'start': 123,
  'end': 140,
  'token_start': 39,
  'token_end': 42,
  'label': 'event_related_to_problem'},
 {'start': 340,
  'end': 354,
  'token_start': 89,
  'token_end': 90,
  'label': 'complication_related_to_problem'}]

In [442]:
last_DC['spans']

[{'start': 2,
  'end': 7,
  'text': 'STEMI',
  'source': '/home/kl533/n2c2/models/plan_subsection_model/model-best-fixed/',
  'input_hash': 1561130997,
  'token_start': 1,
  'token_end': 1,
  'label': 'problem'},
 {'start': 10,
  'end': 54,
  'token_start': 4,
  'token_end': 15,
  'label': 'complication_related_to_problem'},
 {'start': 57,
  'end': 78,
  'token_start': 18,
  'token_end': 24,
  'label': 'complication_related_to_problem'},
 {'start': 81,
  'end': 92,
  'token_start': 27,
  'token_end': 31,
  'label': 'complication_related_to_problem'},
 {'start': 159,
  'end': 208,
  'token_start': 48,
  'token_end': 55,
  'label': 'complication_related_to_problem'},
 {'start': 211,
  'end': 252,
  'token_start': 58,
  'token_end': 66,
  'label': 'complication_related_to_problem'},
 {'start': 255,
  'end': 328,
  'token_start': 69,
  'token_end': 82,
  'label': 'complication_related_to_problem'},
 {'start': 331,
  'end': 362,
  'token_start': 85,
  'token_end': 92,
  'label': 'event_rela

In [441]:
AG_df_hash['spans']

619    [{'start': 0, 'end': 3, 'token_start': 0, 'tok...
Name: spans, dtype: object

In [438]:
DC_df_hash['spans']

Series([], Name: spans, dtype: object)

In [428]:
tmp_df

,AG,DC
interval,,
"[2, 7]",0.0,0.0
"[10, 54]",1.0,1.0
"[57, 78]",-1.0,1.0
"[81, 92]",-1.0,1.0
"[123, 140]",2.0,-1.0
"[159, 208]",-1.0,1.0
"[211, 252]",-1.0,1.0
"[255, 328]",-1.0,1.0
"[331, 362]",1.0,2.0


In [406]:
DC_df_hash

text             # HIV - CD4 300s, VL undetectable.  concern fo...
_input_hash                                              651456513
_task_hash                                              -111059571
_is_binary                                                   False
tokens           [{'text': '#', 'start': 0, 'end': 1, 'id': 0, ...
_view_id                                                ner_manual
spans            [{'start': 2, 'end': 5, 'text': 'HIV', 'source...
answer                                                      accept
_timestamp                                              1655754986
_annotator_id                           n2c2_planSubsection-dc2249
_session_id                             n2c2_planSubsection-dc2249
Name: 379, dtype: object

In [405]:
pd.isna(DC_df_hash['spans'])

array([False, False, False, False])

In [404]:
pd.isna(DC.iloc[2]['spans'])

True

In [415]:
type(DC.iloc[2]['spans']) == float

True

In [380]:
len(DC_df_hash) == 0 or len(AG_df_hash) == 0

True

In [378]:
(pd.isna(DC_df_hash['spans']) == False).bool()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [336]:
# if spans in the df, if spans are not NA and if the len of the span is larger than 0
ann1 = (('spans' in AG_df_hash) and (pd.isna(AG_df_hash['spans']) == False).bool() and (len(AG_df_hash['spans'].values[0]) > 0))

In [346]:
ann2 = (('spans' in DC_df_hash) and (pd.isna(DC_df_hash['spans']) == False).bool() and (len(DC_df_hash['spans'].values[0]) > 0))

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [389]:
# checks if is na and if the len == 0. if any of these two are true continue as there is no annotation for the sample
pd.isna(DC.iloc[2]['spans']) or len(DC.loc[448]['spans']) == 0

True

In [390]:
pd.isna(DC.iloc[2]['spans'])

True

In [363]:
DC.iloc[2]['spans']

nan

In [348]:
DC_df_hash['spans'].any()

False

In [349]:
AG_df_hash['spans'].any()

True

In [339]:
ann1 and ann2

False

In [321]:
pd.isna(DC_df_hash['spans'])

6    True
Name: spans, dtype: bool

In [322]:
DC_df_hash['spans']

6    NaN
Name: spans, dtype: object

In [294]:
len(AG_df_hash['spans'].values[0])

0

In [255]:
tmp_df = pd.DataFrame(lst).set_index('interval').fillna(-1)

In [243]:
tmp_df = tmp_df.set_index('interval')

In [249]:
tmp_df = tmp_df.fillna(-1)

In [256]:
tmp_df

,AG,DC
interval,,
"[2, 5]",0.0,0
"[8, 16]",-1.0,1
"[18, 33]",-1.0,1
"[36, 64]",-1.0,1


In [252]:
from sklearn.metrics import f1_score
f1_score(tmp_df['AG'], tmp_df['DC'], average='weighted')

0.25

In [210]:
x[2]['interval'] = [x[1], x[2]]

In [206]:
a = [{'cs':2, 'ce':5, 'AG': 'problem', 'DC': 'problem'},
 {'cs':8, 'ce':16, 'DC': 'complication_related_to_problem'},
 {'cs':18, 'ce':33, 'DC': 'complication_related_to_problem'},
 {'cs':36, 'ce':64, 'DC': 'complication_related_to_problem'}]
pd.DataFrame(a)

,cs,ce,AG,DC
0,2,5,problem,problem
1,8,16,NaN,complication_related_to_problem
2,18,33,NaN,complication_related_to_problem
3,36,64,NaN,complication_related_to_problem


In [201]:
pd.DataFrame([x[2] for x in merged])

,AG,DC
0,problem,problem
1,NaN,complication_related_to_problem
2,NaN,complication_related_to_problem
3,NaN,complication_related_to_problem


In [190]:
pd.DataFrame(merged)

,0,1,2
0,2,5,"{'AG': 'problem', 'DC': 'problem'}"
1,8,16,{'DC': 'complication_related_to_problem'}
2,18,33,{'DC': 'complication_related_to_problem'}
3,36,64,{'DC': 'complication_related_to_problem'}


In [163]:
from sklearn.metrics import confusion_matrix
import numpy as np
DC_test = ['problem', 'complication_related_to_problem', 'complication_related_to_problem', 'complication', 'complication']
AG_test = ['problem', 'problem', 'complication_related_to_problem', 'complication', 'complication']
cm = confusion_matrix(DC_test, AG_test, labels=['problem', 'complication_related_to_problem', 'complication'])

In [164]:
cm

array([[1, 0, 0],
       [1, 1, 0],
       [0, 0, 2]], dtype=int64)

In [165]:
sum_records = sum(sum(cm))
sum_agreement = np.trace(cm)
P_0 = sum_agreement/sum_records

In [166]:
cm[:,0]

array([1, 1, 0], dtype=int64)

In [169]:
DC_annotation_sum = np.sum(cm, axis=0)
AG_annotation_sum = np.sum(cm, axis=1)
P_e = []
for dc, ag in zip(DC_annotation_sum, AG_annotation_sum):
    P_e.append((dc/sum_records) * (ag/sum_records))
P_e = sum(P_e)
K = (P_0-P_e)/(1-P_e)

In [170]:
K

0.7058823529411765

In [122]:
lst_DC

[{'start': 2,
  'end': 5,
  'text': 'HIV',
  'source': '/home/kl533/n2c2/models/plan_subsection_model/model-best-fixed/',
  'input_hash': 651456513,
  'token_start': 1,
  'token_end': 1,
  'label': 'problem'},
 {'start': 8,
  'end': 16,
  'token_start': 3,
  'token_end': 4,
  'label': 'complication_related_to_problem'},
 {'start': 18,
  'end': 33,
  'token_start': 6,
  'token_end': 7,
  'label': 'complication_related_to_problem'},
 {'start': 36,
  'end': 64,
  'token_start': 10,
  'token_end': 13,
  'label': 'complication_related_to_problem'}]

In [115]:
AG_df_hash['spans'].values

array([list([{'token_start': 1, 'token_end': 1, 'start': 2, 'end': 5, 'text': 'HIV', 'label': 'problem', 'source': '/home/kl533/n2c2/models/plan_subsection_model/model-best-fixed/', 'input_hash': 651456513}])],
      dtype=object)

In [109]:
lst_AG

[{'token_start': 1,
  'token_end': 1,
  'start': 2,
  'end': 5,
  'text': 'HIV',
  'label': 'problem',
  'source': '/home/kl533/n2c2/models/plan_subsection_model/model-best-fixed/',
  'input_hash': 651456513}]

In [108]:
lst_DC

[{'start': 2,
  'end': 5,
  'text': 'HIV',
  'source': '/home/kl533/n2c2/models/plan_subsection_model/model-best-fixed/',
  'input_hash': 651456513,
  'token_start': 1,
  'token_end': 1,
  'label': 'problem'},
 {'start': 8,
  'end': 16,
  'token_start': 3,
  'token_end': 4,
  'label': 'complication_related_to_problem'},
 {'start': 18,
  'end': 33,
  'token_start': 6,
  'token_end': 7,
  'label': 'complication_related_to_problem'},
 {'start': 36,
  'end': 64,
  'token_start': 10,
  'token_end': 13,
  'label': 'complication_related_to_problem'}]

In [69]:
AG[AG['_input_hash'] == 1760190987]['text'].values[0]

'# h/o PE, on coumadin: Remote chance that a transient PE could explain\n   current presentation of hypoxia and hypotension.\n   -continue coumadin with goal INR [**1-30**]'

In [96]:
AG[AG['_input_hash'] == 1760190987]

,text,_input_hash,_task_hash,_is_binary,tokens,_view_id,spans,answer,_timestamp,_annotator_id,_session_id
175,"# h/o PE, on coumadin: Remote chance that a tr...",1760190987,-1754486048,False,"[{'text': '#', 'start': 0, 'end': 1, 'id': 0, ...",ner_manual,"[{'start': 98, 'end': 105, 'token_start': 21, ...",accept,1655226936,n2c2_planSubsection-asg64,n2c2_planSubsection-asg64


In [79]:
AG[AG['_input_hash'] == 1760190987]['spans'].values[0]

[{'start': 98,
  'end': 105,
  'token_start': 21,
  'token_end': 21,
  'label': 'complication_related_to_problem'},
 {'start': 110,
  'end': 121,
  'token_start': 23,
  'token_end': 23,
  'label': 'complication_related_to_problem'}]

In [71]:
DC[DC['_input_hash'] == 1760190987]['text'].values[0]

'# h/o PE, on coumadin: Remote chance that a transient PE could explain\n   current presentation of hypoxia and hypotension.\n   -continue coumadin with goal INR [**1-30**]'

In [80]:
DC[DC['_input_hash'] == 1760190987]['spans'].values[0]

[{'start': 2, 'end': 8, 'token_start': 1, 'token_end': 4, 'label': 'problem'},
 {'start': 13,
  'end': 21,
  'token_start': 7,
  'token_end': 7,
  'label': 'complication_related_to_problem'},
 {'start': 98,
  'end': 105,
  'text': 'hypoxia',
  'source': '/home/kl533/n2c2/models/plan_subsection_model/model-best-fixed/',
  'input_hash': 1760190987,
  'token_start': 21,
  'token_end': 21,
  'label': 'event_related_to_problem'},
 {'start': 110,
  'end': 121,
  'text': 'hypotension',
  'source': '/home/kl533/n2c2/models/plan_subsection_model/model-best-fixed/',
  'input_hash': 1760190987,
  'token_start': 23,
  'token_end': 23,
  'label': 'event_related_to_problem'},
 {'start': 126,
  'end': 158,
  'token_start': 26,
  'token_end': 30,
  'label': 'event_related_to_problem'}]

In [51]:
AG.iloc[1]['spans'][4]

IndexError: list index out of range